In [27]:
# ## Imports
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [28]:
def change():

    names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        # "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        # "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        # "REN"   : "ren",
        "RENFIL": "renfil",
        # "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]
        
        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:
        
        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change
        
        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df

In [29]:
# transactionResponse = {}
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         # "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         # "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         # # "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         # "REN"   : "ren",
#         "RENFIL": "renfil",
#         # "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }
# errors = []
# print("Creating key")
# transactionResponse = {"result":{"AAVEV2":{}}}
# ind       = 0
# look_back = 1


In [30]:
## Compaound market historial API

In [31]:
# res  = requests.get("https://api.compound.finance/api/v2/ctoken")
# for ctoken in res.json()['cToken'][6:]:
#     print(ctoken['symbol'],ctoken['token_address'])
#     res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=1633285800&max_block_timestamp=1634149800&num_buckets=40" %ctoken['token_address']);            
    
#     print(res.json())
#     break

In [32]:
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi",  
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

def change():    
    columns = []
    columns.append('timestamp')
    print(names)
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        print("DEBUG 1 ", asset)
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        # print("DEBUG 1 ", data)
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]

        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
#         df['timestamp']  = tt
#     df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:

        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change

        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df[:-10]
    print("CHECK POINT ")
    return df[1:]

change_df = change()

# min_var = 9999999

{'DAI': 'dai', 'TUSD': 'true-usd', 'USDC': 'usd-coin', 'USDT': 'tether', 'ETH': 'ethereum', 'LINK': 'chainlink', 'MKR': 'maker', 'UNI': 'uniswap', 'WBTC': 'wrapped-bitcoin', 'WBTC2': 'wrapped-bitcoin', 'SUSHI': 'sushi', 'YFI': 'yearn-finance', 'AAVE': 'aave', 'BAT': 'basic-attention-token', 'ZRX': '0x', 'REP': 'augur', 'SAI': 'sai'}
DEBUG 1  DAI
DEBUG 1  TUSD
DEBUG 1  USDC
DEBUG 1  USDT
DEBUG 1  ETH
DEBUG 1  LINK
DEBUG 1  MKR
DEBUG 1  UNI
DEBUG 1  WBTC2
DEBUG 1  SUSHI
DEBUG 1  YFI
DEBUG 1  AAVE
DEBUG 1  BAT
DEBUG 1  ZRX
DEBUG 1  REP
DEBUG 1  SAI
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
!mkdir COMPOUND_DATA

mkdir: cannot create directory ‘COMPOUND_DATA’: File exists


In [34]:
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         # "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         # "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         # # "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         # "REN"   : "ren",
#         "RENFIL": "renfil",
#         # "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }

In [35]:
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi", # check -- - - - - -- 
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

In [36]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
for ctoken in res.json()['cToken'][3:]:
    
    print(ctoken['symbol'])
    if ctoken['symbol'] == 'cCOMP':
        continue

    ind += 1
    for key in ctoken.keys():
        try:
            df.at[ind,key ] = ctoken[key]['value']
        except:
            df.at[ind,key ] = ctoken[key]

    

    address   = ctoken['token_address']

    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    prices_df = pd.DataFrame(columns=['timestamp', 'price'])
    timestamp = []
    prices    = []

    for req in request.json()['prices']:
        timestamp.append(int(float(req[0])/1000))
        prices.append(req[1])

    prices_df['timestamp'] = timestamp
    prices_df['price']     = prices

    date= []
    for time in timestamp:
        date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
    prices_df['date']  = date

cWBTC
cMKR
cZRX
cYFI
cSUSHI
cLINK
cBAT
cUNI
cWBTC2
cUSDT
cTUSD
cCOMP
cAAVE
cUSDC
cETH
cDAI


In [37]:
prices_df

,timestamp,price,date
0,1574121600,1.000651,2019-11-19
1,1574208000,0.995411,2019-11-20
2,1574294400,0.991591,2019-11-21
3,1574380800,0.991102,2019-11-22
4,1574467200,0.992850,2019-11-23
...,...,...,...
783,1641772800,1.002857,2022-01-10
784,1641859200,1.001071,2022-01-11
785,1641945600,0.999613,2022-01-12
786,1642032000,1.001430,2022-01-13


In [38]:

# PLATFORM: AAVE_V2, Compound


In [39]:
# !mkdir COMPOUND_DATA

In [40]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
# MASTER_DATE = '2021-12-20'
MASTER_DATE = "2022-01-13"
for ctoken in res.json()['cToken']:
    try:
        ind += 1
        for key in ctoken.keys():
            try:
                df.at[ind,key ] = ctoken[key]['value']
            except:
                df.at[ind,key ] = ctoken[key]


        address   = ctoken['token_address']

        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        prices_df = pd.DataFrame(columns=['timestamp', 'price'])
        timestamp = []
        prices    = []

        for req in request.json()['prices']:
            timestamp.append(int(float(req[0])/1000))
            prices.append(req[1])

        prices_df['timestamp'] = timestamp
        prices_df['price']     = prices

        date= []
        for time in timestamp:
            date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
        prices_df['date']  = date

#         datestamp = '2021-12-14'
        datestamp = MASTER_DATE
        max_timstamp = int(datetime.strptime(datestamp, '%Y-%m-%d').timestamp())
        SUPPLY_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        BORROW_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        DF = pd.DataFrame()
        BIND = 0
        SIND = 0
        for _ in range(5):
            min_timestamp = max_timstamp  -  30 * 86400 
            res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=50" % ( address,min_timestamp, max_timstamp));    
            max_timstamp = max_timstamp   - 30 * 86400 
            temp = res.json()


            for tem in temp['total_borrows_history']:
                BORROW_DF.loc[BIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]                     
                BIND += 1


            for tem in temp['total_supply_history']:
                SUPPLY_DF.loc[SIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]
                SIND += 1                      


            print("***")

        BORROW_DF = BORROW_DF.sort_values(['block_timestamp'])
        BORROW_DF['DATE'] = BORROW_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        BORROW_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)
        SUPPLY_DF = SUPPLY_DF.sort_values(['block_timestamp'])
        SUPPLY_DF['DATE'] = SUPPLY_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        SUPPLY_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

        SUPPLY_DF['supply_tokens'] = pd.to_numeric(SUPPLY_DF['value']) *  float(temp['exchange_rates'][-1]['rate'])


        BORROW_DF = BORROW_DF[::-1]
        SUPPLY_DF = SUPPLY_DF[::-1]
        DF['timestamp']    = BORROW_DF['block_timestamp']
        DF['BORROW']       = BORROW_DF['value']
        DF['SUPPLY']       = SUPPLY_DF['supply_tokens']
        DF['DATE']         = BORROW_DF['DATE']
        DF['SUPPLY_VALUE'] = SUPPLY_DF['value']


        assigned_price = []
        for indW in DF['DATE'] :
            a = prices_df[prices_df['date']==str(indW)]
            assigned_price.append(a['price'].values[0])

        DF['prices'] = assigned_price

        DF['total_supply'] =  DF['SUPPLY'] * DF['prices']
        DF['total_borrow'] =  pd.to_numeric(DF['BORROW']) * pd.to_numeric(DF['prices'])
        DF['POSITION']     =  DF['total_borrow'] - DF['total_supply']
        DF['POSITION($)']  =  DF['POSITION'] / pow(10,9)
        DF.to_csv('COMPOUND_DATA/DF_'+ctoken['symbol'][1:] + '.csv')



        print("**********************", DF.shape)
    except Exception as e:
        print("ERROR ", e, ctoken['symbol'])
    

***
***
***
***
***
********************** (65, 10)
***
***
***
***
***
ERROR  index 0 is out of bounds for axis 0 with size 0 cSAI
ERROR  'USDP' cUSDP
***
***
***
***
***
********************** (144, 10)
***
***
***
***
***
********************** (135, 10)
***
***
***
***
***
********************** (148, 10)
***
***
***
***
***
********************** (143, 10)
***
***
***
***
***
********************** (147, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (142, 10)
ERROR  'COMP' cCOMP
***
***
***
***
***
ERROR  [Errno 13] Permission denied: 'COMPOUND_DATA/DF_AAVE.csv' cAAVE
***
***
***
***
***
********************** (150, 10)
***
***
***


KeyboardInterrupt: 

In [ ]:
fils = os.listdir('COMPOUND_DATA/')

In [ ]:
cols = []
for fil in fils:
    try:
        aa = fil.split('.')[0].split('_')
    #     print(aa[1])
        cols.append(aa[1])
    except:
        pass

In [ ]:
# df

In [ ]:
FINAL ={}
for key in cols:
    try:
        df = pd.read_csv('COMPOUND_DATA/'+'DF_'+key+'.csv')
        df = df.sort_values(['timestamp'])
#         print(df.head())
        df = df[::-1]
        FINAL[key] = {'Assets': df['POSITION($)'],
                      'date':df['DATE'],
                      'MARKET':df['total_supply'],
                      'BORROW':df['total_borrow']
                     }
    
    except Exception as e:
        print(e, key)


In [ ]:
ff = []
for f in FINAL['ETH']['date']:
    ff.append(f)

In [ ]:
FINAL_DF = pd.DataFrame(columns= cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['Assets'][i])
        except:
            assets.append([])
    
    FINAL_DF.loc[ind] = assets
    ind += 1

In [ ]:
FINAL_DF['DATE'] = ff[:100]
FINAL_DF['timestamp'] = FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
FINAL_DF = FINAL_DF.sort_values('timestamp')
FINAL_DF = FINAL_DF[::-1]

In [ ]:
FINAL_DF

In [ ]:
change_df#[344:345]

In [ ]:
change_df[0:1]

In [ ]:
VAR = []
VAR_DATE = []
for ind, ROW in enumerate(FINAL_DF.iterrows()):
    REQUIRED_10 = []
    for required_row in change_df[ind:ind+365].iterrows():
        SUM_10 = []
        for key in ROW[1].keys()[:-3]:
            if key == 'WBTC2':
                key = 'WBTC'
            if isinstance(ROW[1][key], list):
                ROW[1][key] = 0
            SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
        REQUIRED_10.append(sum(SUM_10))
    VAR.append(sorted(REQUIRED_10, key=float)[3])
    VAR_DATE.append(change_df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])

In [ ]:
VAR_DATE

In [ ]:
POS_VAR = [-x for x in VAR]

In [ ]:
plt.plot(POS_VAR)

In [ ]:
max(VAR), min(VAR)

In [ ]:
max(VAR), min(VAR)

In [ ]:
# textfile = open("a_file.txt", "w")
# for element in VAR:
#     textfile.write(element + "\n")
# textfile.close()

In [ ]:
# ff = []
# for f in FINAL['ETH']['date']:
#     ff.append(f)

MARKET_FINAL_DF = pd.DataFrame(columns= cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['MARKET'][i])
        except Exception as e:
            assets.append([])
    MARKET_FINAL_DF.loc[ind] = assets
    ind += 1

In [ ]:
for i in range(MARKET_FINAL_DF.shape[0]):
    for j in range(1,MARKET_FINAL_DF.shape[1]-1):
        if isinstance(MARKET_FINAL_DF.iloc[i,j], list):
            print(MARKET_FINAL_DF.iloc[i,j])
            MARKET_FINAL_DF.iloc[i,j] = 0
            
        if 'e' in str(MARKET_FINAL_DF.iloc[i,j]):
            MARKET_FINAL_DF.iloc[i,j] = 0

In [ ]:
MARKET_FINAL_DF['DATE'] = ff[:100]
MARKET_FINAL_DF['timestamp'] = MARKET_FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
MARKET_FINAL_DF = MARKET_FINAL_DF.sort_values('timestamp')
MARKET_FINAL_DF = MARKET_FINAL_DF[::-1]

In [ ]:
MARKET_FINAL_DF[MARKET_FINAL_DF.columns[:-2]].sum(axis=1)

In [ ]:
# ff = []
# for f in FINAL['ETH']['date']:
#     ff.append(f)

BORROW_FINAL_DF = pd.DataFrame(columns=cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['BORROW'][i])
        except:
            assets.append([])
    
    BORROW_FINAL_DF.loc[ind] = assets
    ind += 1

In [ ]:
for i in range(BORROW_FINAL_DF.shape[0]):
    for j in range(1,BORROW_FINAL_DF.shape[1]-1):
        if isinstance(BORROW_FINAL_DF.iloc[i,j], list):
            print(BORROW_FINAL_DF.iloc[i,j])
            BORROW_FINAL_DF.iloc[i,j] = 0
            
        if 'e' in str(BORROW_FINAL_DF.iloc[i,j]):
            BORROW_FINAL_DF.iloc[i,j] = 0

In [ ]:
BORROW_FINAL_DF['DATE'] = ff[:100]
BORROW_FINAL_DF['timestamp'] = BORROW_FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
BORROW_FINAL_DF = BORROW_FINAL_DF.sort_values('timestamp')
BORROW_FINAL_DF = BORROW_FINAL_DF[::-1]

In [ ]:
BORROW_FINAL_DF[BORROW_FINAL_DF.columns[:-2]].sum(axis=1)

In [ ]:
BORROW_FINAL_DF.head()

In [ ]:
MARKET_FINAL_DF.head()

In [ ]:
len(BORROW_FINAL_DF.columns)

In [ ]:
len(BORROW_FINAL_DF)

In [ ]:
CONS_DATA = pd.DataFrame(columns=["date", "Liabilities", "Assets", "VAR", "VAR_DATE", "timestamp"])

In [ ]:
CONS_DATA['date'] = MARKET_FINAL_DF['DATE'] 

In [ ]:
CONS_DATA['timestamp'] = MARKET_FINAL_DF['timestamp'] 

In [ ]:
CONS_DATA['Assets'] = BORROW_FINAL_DF[BORROW_FINAL_DF.columns[:-2]].sum(axis=1)

In [ ]:
CONS_DATA['Liabilities'] = MARKET_FINAL_DF[MARKET_FINAL_DF.columns[:-2]].sum(axis=1)

In [ ]:
CONS_DATA["VAR"] = VAR

In [ ]:
CONS_DATA["VAR_DATE"] = VAR_DATE

In [ ]:
# Saving to CSV
CONS_DATA.to_csv("compound-historical-dec-20.csv", index=False)